# Check and Install Required Packages
Ensure all required packages are installed, including ipykernel.

In [ ]:
# Ensure ipykernel and other required packages are installed
import sys
import subprocess
import pkg_resources

required = {'ipykernel', 'pandas', 'numpy', 'matplotlib', 'scipy', 'scikit-learn'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    print(f'Installing missing packages: {missing}')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])

# Import Required Libraries
Import necessary libraries such as pandas, numpy, matplotlib, and sklearn.

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Load Dataset
Load the ratings and movies datasets using pandas.

In [ ]:
# Load Dataset
ratings = pd.read_csv(r'path_to_ratings.csv')
movies = pd.read_csv(r'path_to_movies.csv')

# Display the first few rows of the datasets
ratings.head(), movies.head()

# Create User-Movie Matrix
Create a sparse user-movie matrix using the ratings data.

In [ ]:
# Create a sparse user-movie matrix
user_movie_matrix_sparse = csr_matrix((ratings['rating'], (ratings['userId'], ratings['movieId'])))

# Display the shape of the user-movie matrix
user_movie_matrix_sparse.shape

# Build Recommendation Model
Build a recommendation model using NearestNeighbors from sklearn.

In [ ]:
# Build Recommendation Model

# Calculate cosine similarity between users using NearestNeighbors for efficiency
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_movie_matrix_sparse)

# Function to recommend movies
def recommend_movies(user_id, model_knn, user_movie_matrix_sparse, movies, top_n):
    print(f'Generating recommendations for user {user_id}...')
    # Get the ratings of the target user
    user_ratings = user_movie_matrix_sparse[user_id].toarray().flatten()

    # Find movies that the target user has not seen (rating is 0)
    unseen_movies = (user_ratings == 0).nonzero()[0]

    # Get similar users and their similarity scores
    distances, indices = model_knn.kneighbors(user_movie_matrix_sparse[user_id], n_neighbors=top_n)
    similar_users = pd.Series(distances.flatten(), index=indices.flatten())
    similar_users = similar_users[similar_users > 0]

    # Calculate the weighted sum of ratings from similar users
    weighted_sum = user_movie_matrix_sparse[similar_users.index].T.dot(1 - similar_users.values)

    # Calculate the sum of similarity scores
    similarity_sum = (1 - similar_users.values).sum()

    # Calculate recommendations by dividing weighted sum by similarity sum
    with np.errstate(divide='ignore', invalid='ignore'):
        recommendations = np.true_divide(weighted_sum, similarity_sum)
        recommendations[~np.isfinite(recommendations)] = 0  # set infinities and NaNs to 0

    # Filter recommendations to only include unseen movies and sort them
    recommendations = pd.Series(recommendations[unseen_movies], index=unseen_movies).sort_values(ascending=False)

    # Merge recommendations with movie data to get the release year
    recommendations = recommendations.reset_index().rename(columns={'index': 'movieId'}).merge(movies[['movieId', 'title']], on='movieId')

    return recommendations.head(top_n)

# Example usage
target_user = 1  # Example user ID
top_n = 10  # Number of recommendations
recommended_movies = recommend_movies(target_user, model_knn, user_movie_matrix_sparse, movies, top_n)
recommended_movies[['movieId', 'title']]